In [1]:
import sys; sys.path.insert(0,'..')
from hashstash import *
logger.setLevel(logging.INFO)
from hashstash.profilers.engine_profiler import *

In [8]:
# HashStash(engine='memory').profiler.profile_simple(_force=True, data_types=['pandas_df'], size=1_000, num_proc=5)

In [9]:
# # df=HashStash(engine='dataframe').profile(num_proc=10, size=10_000, data_types=['pandas_df'], iterations=10000)
# # df
# HashStashProfiler.run_profiles(iterations=10000, size=1_000, num_procs=[1,8], progress_inner=False,progress=True, data_types=['pandas_df'])

In [10]:
# df=HashStashProfiler.run_profiles(iterations=10000, size=1_000, num_procs=[1], progress_inner=True,progress=True, data_types=['pandas_df'])
# df=HashStashProfiler.run_profiles(iterations=10000, size=1_000, num_procs=[1], progress_inner=True,progress=True, data_types=['dict'])

In [11]:
def query(obj):
    d=obj.get('kwargs')
    try:
        if d['serializer'] == 'pickle': return False
        if d['num_proc'] != 1: return False
        if d['size']<1000: return False
        # if d['size']<1000: return False
    except KeyError:
        return False
    return True

f=HashStashProfiler.profile
fstash = f.stash
o = []
import pandas as pd
for key,val in fstash.query(query,return_vals=True):
    o.append(val)
odf=pd.concat(o) if o else pd.DataFrame()
odf

[1.80s] querying by key = query and value = bool: 100%|██████████| 18/18 [00:00<00:00, 134.21it/s]


,Iteration,Num Proc,Cumulative Time (s),Cumulative Size (B),Cumulative Rate (it/s),Cumulative Speed (MB/s),Engine,Serializer,Encoding,Data Type,Size (B),Raw Size (B),Cached Size (B),Compression Ratio (%),Operation,Time (s),Rate (it/s),Speed (MB/s)
0,0,1,0.114432,14244,0.000000,0.118708,memory,hashstash,zlib+b64,DataFrame,10000,14244,11008,77.281662,Serialize,0.014242,70.214008,0.953797
1,0,1,0.114434,14244,8.738661,0.118706,memory,hashstash,zlib+b64,DataFrame,10000,14244,11008,77.281662,Deserialize,0.001816,550.650387,7.480110
2,0,1,0.114435,14244,17.476995,0.118705,memory,hashstash,zlib+b64,DataFrame,10000,14244,11008,77.281662,Encode,0.000421,2376.376204,32.281020
3,0,1,0.114437,14244,26.215328,0.118704,memory,hashstash,zlib+b64,DataFrame,10000,14244,11008,77.281662,Decode,0.000107,9341.434298,126.895323
4,0,1,0.114437,14244,34.953480,0.118703,memory,hashstash,zlib+b64,DataFrame,10000,14244,11008,77.281662,Set,0.027786,35.989326,0.488884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,99,1,7.912302,1491803,87.837901,0.179808,lmdb,jsonpickle,zlib+b64,DataFrame,10000,12856,9544,74.237710,Encode,0.000202,4951.952774,60.713105
696,99,1,7.912303,1491803,87.964273,0.179808,lmdb,jsonpickle,zlib+b64,DataFrame,10000,12856,9544,74.237710,Decode,0.000088,11335.956757,138.983784
697,99,1,7.912304,1491803,88.090648,0.179808,lmdb,jsonpickle,zlib+b64,DataFrame,10000,12856,9544,74.237710,Set,0.015739,63.535621,0.778974
698,99,1,7.912305,1491803,88.217023,0.179808,lmdb,jsonpickle,zlib+b64,DataFrame,10000,12856,9544,74.237710,Get,0.000063,15887.515152,194.787879


In [14]:
df=HashStashProfiler.run_profiles(iterations=100, size=10_000, num_procs=[1], progress_inner=True,progress=True, data_types=['pandas_df'])


[4.00s] [1x]: 100%|██████████| 18/18 [00:00<00:00, 139.35it/s]


In [16]:
# import pandas as pd
# import plotnine as p9
# p9.options.figure_size = (12,8)

# def plot_iterations(df):
#     df['Iteration Group'] = df['Iteration']# // 1000 * 1000


#     # df=df[df.Operation.isin({'Total'})]
#     df=df[df.Operation.isin({'Total'})]
#     df=df[df['Data Type'].isin({'DataFrame'})]
#     # df=df[df.Serializer.isin({'hashstash','jsonpickle'})]
#     # df = df[(df['Num Proc'].isin({1}))]
#     # df = df[(df['Data Type'].isin({'DataFrame'}))]
#     df = df.query('1000<Iteration')

#     figdf = df.groupby(['Iteration Group','Engine','Data Type', 'Size (B)']).median(numeric_only=True).reset_index()
#     figdf['Cumulative Size (MB)'] = figdf['Cumulative Size (B)'] / 1024 / 1024
#     figdf['Cumulative Size (GB)'] = figdf['Cumulative Size (MB)'] / 1024
#     # figdf = figdf[(figdf['Cumulative Size (MB)'] > 1) & (figdf['Cumulative Size (MB)'] < 5)]
#     # engine_speed_avg = df.groupby('Engine')['Speed (MB/s)'].mean().sort_values(ascending=False)
#     # figdf['Engine'] = pd.Categorical(figdf['Engine'], categories=engine_speed_avg.index, ordered=True)
#     # fig = p9.ggplot(figdf, p9.aes(x='Cumulative Size (MB)',y='Cumulative Speed (MB/s)',color='Engine'))
#     fig = p9.ggplot(figdf, p9.aes(x='Iteration Group',y='Cumulative Rate (it/s)',color='Engine'))
#     # fig = p9.ggplot(figdf, p9.aes(x='Cumulative Size (GB)',y='Cumulative Speed (MB/s)',color='Engine'))
#     # fig = p9.ggplot(figdf, p9.aes(x='Iteration Group',y='Cumulative Time (s)',color='Engine'))
#     fig+=p9.geom_line()
#     # fig+=p9.geom_smooth()
#     fig+=p9.geom_text(p9.aes(label='Engine'), data=pd.concat(dfx.iloc[-1:] for g,dfx in figdf.groupby(['Engine','Data Type', 'Size (B)'])))
#     # fig+=p9.geom_point()
#     fig+=p9.theme_classic()
#     # fig+=p9.facet_wrap('Data Type', scales='free',ncol=1)
#     # fig+=p9.facet_wrap('Serializer', nrow=1)
#     fig+=p9.facet_wrap('Size (B)', nrow=1)
#     # fig+=p9.facet_grid('Data Type ~ Size (B)',scales='free')
#     # fig+=p9.scale_y_log10()
#     # fig+=p9.scale_x_log10()
#     return fig


# plot_iterations(df)

ValueError: No objects to concatenate

In [ ]:
import pandas as pd
import plotnine as p9


def plot_num_proc():
    df['Iteration Group'] = df['Iteration'] // 100
    figdf = df[df.Operation=='Total'].groupby(['Engine','Num Proc']).median(numeric_only=True).reset_index()
    engine_speed_avg = df.groupby('Engine')['Speed (MB/s)'].mean().sort_values(ascending=False)
    figdf['Engine'] = pd.Categorical(figdf['Engine'], categories=engine_speed_avg.index, ordered=True)
    fig = p9.ggplot(figdf, p9.aes(x='Num Proc',y='Rate (it/s)',color='Engine',label='Engine'))
    fig+=p9.geom_line(p9.aes(group='Engine'))
    # fig+=p9.geom_point()
    fig+=p9.geom_text(size=8, ha='right')
    fig+=p9.theme_classic()
    fig+=p9.scale_y_log10()
    return fig


plot_num_proc()

In [ ]:
# f = HashStashProfiler(HashStash()).profile
f = HashStashProfiler.profile
fstash = f.stash


fstash

import pandas as pd

In [ ]:
# df=HashStashProfiler.run_profiles(iterations=2000, size=1_000, num_proc=1, num_procs=[4], progress_inner=False, progress=True, data_types=['pandas_df'])
# engine_speed_avg = df[df.Operation == 'Total'].groupby('Engine')['Speed (MB/s)'].mean().sort_values(ascending=False)
# df['Engine'] = pd.Categorical(df['Engine'], categories=engine_speed_avg.index, ordered=True)
# df

In [ ]:
import plotnine as p9


# def plot_iterations():
#     figdf = df.groupby(['Iteration','Engine']).median(numeric_only=True).reset_index()

#     fig = p9.ggplot(figdf, p9.aes(x='Iteration',y='Rate (it/s)',color='Engine'))
#     fig+=p9.geom_line()
#     return fig

# plot_iterations()

In [ ]:
df.groupby(['Data Type','Engine']).size()

In [ ]:
def plot_data_types():
    figdf = df.groupby(['Data Type','Engine']).median(numeric_only=True).reset_index()
    
    fig = p9.ggplot(figdf, p9.aes(x='Engine',y='Rate (it/s)',color='Data Type',group='Data Type'))
    fig+=p9.geom_point()
    fig+=p9.geom_line()
    fig+=p9.scale_y_log10()
    return fig

plot_data_types()

In [ ]:
# fstash.df

In [ ]:
# # f(iterations=100,num_proc=5, num_procs=[1])
# f(iterations=1000,num_proc=1, num_procs=[1,2,4,8], progress_inner=True, engines = [e for e in ENGINES if e not in {'shelve'}])
# # f(iterations=100,num_proc=1, num_procs=[5], engines=['memory','pairtree'])

# # f(iterations=100,num_proc=1, num_procs=[5], _force=False, engines=['memory','pairtree'])

In [ ]:
df[(df.Operation=='Total') & (df['Data Type']=='DataFrame')].groupby(['Engine','Serializer']).median(numeric_only=True).sort_values('Speed (MB/s)', ascending=False)[['Speed (MB/s)','Time (s)']]

In [ ]:
# sdf=df[df.Operation.isin({'Total'}) & (df['Serializer']=='hashstash') & (df['Engine']!='memory')]
# sdf=df[df.Operation.isin({'Total'}) & (df['Serializer']=='hashstash') & (df['Engine']!='memory') & (df['Data Type']!='DataFrame')]
df.groupby(['Engine','Serializer','Num Proc']).median(numeric_only=True).sort_values('Rate (it/s)', ascending=False)[['Speed (MB/s)','Time (s)','Rate (it/s)']].head(25)


In [ ]:
df_avg = df.groupby(['Engine','Serializer','Num Proc','Operation']).median(numeric_only=True).reset_index()
df_avg

In [ ]:
df_avg['Num Proc'].value_counts()

In [ ]:
# import plotnine as p9

# df_avg = df_avg[df_avg.Operation.isin({'Get','Set','Total'})]
# # df_avg = df_avg[df_avg.Serializer.isin({'hashstash','jsonpickle'})]

# import pandas as pd

# # Calculate the average speed for each engine
# engine_speed_avg = df_avg[df_avg.Operation == 'Total'].groupby('Engine')['Speed (MB/s)'].mean().sort_values(ascending=False)

# # Convert Engine to a categorical type with categories sorted by average speed
# df_avg['Engine'] = pd.Categorical(df_avg['Engine'], categories=engine_speed_avg.index, ordered=True)


# fig = p9.ggplot(df_avg, p9.aes(x='Engine', y='Speed (MB/s)'))
# fig += p9.geom_boxplot() 
# fig += p9.geom_point(p9.aes(color='Operation', shape='Serializer'),size=3) + p9.theme_classic() + p9.scale_y_log10()
# fig

In [ ]:
import pandas as pd
import plotnine as p9
p9.options.figure_size = (12,8)
df_avg = df.groupby(['Engine','Num Proc','Serializer','Operation']).median(numeric_only=True).reset_index()
figdf=df_avg[df_avg.Operation.isin({'Get','Set','Serialize','Deserialize'})]

engine_speed_avg = df.groupby('Engine')['Time (s)'].median().sort_values(ascending=True)
# # Convert Engine to a categorical type with categories sorted by average speed
figdf['Engine'] = pd.Categorical(figdf['Engine'], categories=engine_speed_avg.index, ordered=True)

fig = (p9.ggplot(figdf, p9.aes(x='Engine', y='Time (s)',color='Serializer'))
       # + p9.geom_boxplot()
       + p9.geom_point(size=3)
       + p9.geom_line(p9.aes(group='Serializer'))
       + p9.facet_wrap('Operation', ncol=2,scales='free')
       + p9.theme_classic()
       + p9.scale_y_log10()
       
       )
fig

In [ ]:
import pandas as pd
import plotnine as p9
figdf=df_avg[df_avg.Operation.isin({'Total'})]
# # Convert Engine to a categorical type with categories sorted by average speed
figdf['Engine'] = pd.Categorical(figdf['Engine'], categories=engine_speed_avg.index, ordered=True)

fig = (p9.ggplot(figdf, p9.aes(x='Engine', y='Speed (MB/s)'))
       + p9.geom_boxplot()
       + p9.geom_point(p9.aes(color='Operation', shape='Serializer'), size=3)
       + p9.facet_wrap('Num Proc', ncol=1)
       + p9.theme_classic()
       + p9.scale_y_log10())
fig

In [ ]:
figdf = df.query('Operation=="Total"').groupby(['Engine','Serializer','Size']).median(numeric_only=True).reset_index()



fig = (p9.ggplot(figdf, p9.aes(x='Engine', y='Speed (MB/s)'))
       + p9.geom_boxplot()
       + p9.geom_point(p9.aes(color='Data Type', shape='Serializer'), size=3)
       + p9.theme_classic()
       # + p9.scale_y_log10()
)
fig

In [ ]:
df['Data Type'].value_counts()

In [ ]:
# df['Num Proc'] = df['Num Proc'].apply(lambda x: f'{int(x):02}x')
df['Size (KB)'] = df['Raw Size (MB)'].apply(lambda x: int(x*1024))
df['Size (B)'] = df['Raw Size (MB)'].apply(lambda x: int(x*1024*1024))

import math

def group_by_magnitude(size_bytes):
    if size_bytes == 0:
        return '0 B'
    
    magnitude = int(math.log10(size_bytes))
    unit_index = magnitude // 3
    units = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    
    if unit_index >= len(units):
        return f'{size_bytes} B'  # Fallback for extremely large sizes
    
    unit = units[unit_index]
    # return unit
    scaled_size = size_bytes / (1024 ** unit_index)
    
    if magnitude % 3 == 0:
        return f'1 {unit}'
    elif magnitude % 3 == 1:
        return f'10 {unit}'
    else:
        return f'100 {unit}'

# Apply the function to create the new column
# df['Size Name'] = df['Size (B)'].apply(lambda x: 10**(round(math.log10(x))//1*1))
df['Size Name'] = df['Size (B)'].apply(lambda x: group_by_magnitude(x))
# df['Size Name'] = df['Size (B)'].apply(lambda x: x//1000*1000)
# df['Size Name'] = df['Size (B)'].apply(lambda x: "B" if x<1000 else "KB" if x<1000000 else "MB")
df['Size Name'].value_counts()
df=df[df['Size Name'].isin({'1 KB'})]

In [ ]:
def get_speed_stats(group_by:list=None, operations:list=None, carry_over_cols:list=('Speed (MB/s)', 'Time (s)'), also_cols=('Raw Size (MB)',)):
    global df
    # df = df.reset_index().replace({'':np.nan})
    if operations:
        df = df[df['Operation'].isin(operations)]
    
    # Group by Engine and Operation, calculate median for numeric columns
    gby = ([] if not group_by else list(group_by)) + ['Operation']
    grouped = df.groupby(gby).median(numeric_only=True)
    
    # Reset index to make Operation a column
    grouped = grouped.reset_index()
    
    # Pivot the table to get speeds and carried over columns as separate columns
    pivot_values = list(carry_over_cols)
    pivoted = grouped.pivot(index=list(group_by)+list(also_cols), columns='Operation', values=pivot_values)
    
    # Flatten column names
    # pivoted.columns = [f'{col[1]} {col[0]}' if col[0] in carry_over_cols else f'{col[1]} (MB/s)' for col in pivoted.columns]
    pivoted.columns = [f'{col[1]} {col[0].split()[-1]}' for col in pivoted.columns]
    
    # Sort by Total speed descending
    odf = pivoted.reset_index()#.sort_values('Total (MB/s)', ascending=False)
    for c in odf:
        if c.endswith('(s)'):
            odf[f'{c.replace("(s)","(it/s)")}'] = 1/odf[c]
    odf['GetSet (s)'] = odf['Get (s)'] + odf['Set (s)']
    odf['GetSet (it/s)'] = 1/odf['GetSet (s)']
    odf['Group Name'] = ['+'.join(row[x] for x in group_by) for i,row in odf.iterrows()]
    return odf.set_index('Group Name').sort_values('Total (MB/s)', ascending=False)

get_speed_stats(['Engine','Serializer'])['Total (MB/s)']

In [ ]:
# Call the function and display results
get_speed_stats(group_by=['Engine','Size Name'])

In [ ]:
import plotnine as p9
from plotnine import *

def plot_speed_comparison(df, group_by=['Engine'], x_col='Write', y_col='Read', color=None, log=False, facet=None):
    p9.options.figure_size = (8, 6)
    # Prepare the data
    if color and not color in group_by: group_by.append(color)
    if facet and not facet in group_by: group_by.append(facet)
    speed_df = get_speed_stats(group_by=group_by)
    speed_df = speed_df.reset_index()
    
    # Extract the specific operations we want to compare
    # cols = [*group_by]
    # if not x_col in cols: cols.append(x_col)
    # if not y_col in cols: cols.append(y_col)
    plot_df = speed_df #[cols]
    
    # Create a label column that combines all group_by columns
    plot_df['Label'] = plot_df[group_by].apply(lambda row: ' '.join(row.values[:1].astype(str)), axis=1)
    
    # Create the plot
    aes_params = {'x': x_col, 'y': y_col, 'label': 'Label'}
    if color:
        aes_params['color'] = color
    
    plot = (
        ggplot(plot_df, aes(**aes_params))
        # + geom_point(size=3)
        + geom_text(aes(label='Label'), size=8)
        + theme_classic()
        + labs(title=f'{y_col} vs {x_col} Speed Comparison')
            #    x=f'{x_operation} Speed (MB/s)',
            #    y=f'{y_operation} Speed (MB/s)')
    )

    if log:
        plot += scale_x_log10()
        plot += scale_y_log10()
        # plot += labs(x=f'{x_operation} Speed (MB/s) - Log Scale',
                    #  y=f'{y_operation} Speed (MB/s) - Log Scale')

    if facet:
        plot += facet_wrap(facet,ncol=2, scales='free')

    return plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Read (it/s)', y_col='Write (it/s)', log=True)
comparison_plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Get (it/s)', y_col='Set (it/s)', log=True)
comparison_plot

In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Engine'], x_col='Get (it/s)', y_col='Set (it/s)', log=True, color='Data Type')
comparison_plot


In [ ]:
comparison_plot = plot_speed_comparison(df, group_by=['Data Type','Serializer'], x_col='Serialize (it/s)', y_col='Deserialize (it/s)',log=True, color='Serializer')
comparison_plot

In [ ]:
import plotnine as p9
comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='GetSet (it/s)', log=False, color='Engine') + p9.geom_line()
comparison_plot


In [ ]:
# import plotnine as p9
# comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Total (MB/s)', log=False, color='Engine') + p9.geom_line()
# comparison_plot


In [ ]:
import plotnine as p9
comparison_plot = plot_speed_comparison(df, group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Write (it/s)', log=True, color='Engine') + p9.geom_line() + p9.scale_x_continuous()
comparison_plot


In [ ]:
# import plotnine as p9
# comparison_plot = plot_speed_comparison(df[df.Serializer=='hashstash'], group_by=['Engine','Num Proc'], x_col='Num Proc', y_col='Set (MB/s)', log=True, color='Engine',facet='Size Name') + p9.geom_line()
# comparison_plot
